In [ ]:
import opendatasets as od
import os

import pandas as pd
import numpy as np

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
# od.download('https://www.kaggle.com/c/rossmann-store-sales/overview')

In [ ]:
os.listdir('./rossmann-store-sales')

In [ ]:
train_df = pd.read_csv('./rossmann-store-sales/train.csv', low_memory=False)
store_df = pd.read_csv('./rossmann-store-sales/store.csv')
test_df = pd.read_csv('./rossmann-store-sales/test.csv')
submission_df = pd.read_csv('./rossmann-store-sales/sample_submission.csv')

In [ ]:
train_df

In [ ]:
test_df

In [ ]:
submission_df

In [ ]:
store_df

In [ ]:
train_df = train_df.merge(store_df, how='left', on='Store')
test_df = test_df.merge(store_df, how='left', on='Store')

In [ ]:
train_df

In [ ]:
train_df.columns

In [ ]:
sns.barplot(train_df.Customers, train_df.Sales);